In [162]:
import os
import time
import datetime
import pathlib
import glob

In [99]:
# csgo_path = "C:\Program Files (x86)\Steam\steamapps\common\Counter-Strike Global Offensive"

csgo_path = os.path.join(
    '/Users', 'necuk', 'Library', 'Application Support', 'Steam', 'steamapps',
    'common', 'Counter-Strike Global Offensive'
)

os.listdir(csgo_path)

['csgo.sh',
 '.DS_Store',
 'bin',
 'steam_appid.txt',
 'installscript.vdf',
 'platform',
 'chrome.pak',
 'csgo_osx64',
 'thirdpartylegalnotices.doc',
 'WINDOWSTEMPDIR_FONTCONFIG_CACHE',
 'csgo',
 'EmptySteamDepot']

In [100]:
import shutil
import os
import time

import pyautogui

In [173]:
class Parser:
    def __init__(self, csgo_path, output_folder='voice_output', parsing_time=0.3, file_size_alert=800000):
        # csgo stops storing voice data when file size reaches 1048620
        self.csgo_path = csgo_path
        self.voice_file = os.path.join(csgo_path, 'voice_decompressed.wav')
        self.file_size_alert = file_size_alert
        self.counter = 0
        self.prev_file_size = 0
        self.parsing_time = parsing_time
        self.output_folder = os.path.join(csgo_path, output_folder)
        os.makedirs(self.output_folder, exist_ok=True)
#         self.empty_output_folder()
    
    
    def empty_output_folder(self):
        os.makedirs(self.output_folder, exist_ok=True)
        files = glob.glob(f'{self.output_folder}/*')
        for f in files:
            os.remove(f)
    
    
    def fast_forward(self):
        pyautogui.press('f1')
       
    
    def a_bit_faster(self):
        pyautogui.press('f3')
    
    
    def normal_speed(self):
        pyautogui.press('f4')
        
        
    def pause(self):
        pyautogui.press('f5')
        
        
    def resume(self):
        pyautogui.press('f6')
        
        
    def start_record(self):
        pyautogui.keyDown('c')
        
        
    def stop_record(self):
        self.pause()
        pyautogui.keyUp('c')
        time.sleep(0.02)
        
        size = os.path.getsize(self.voice_file)
        print(f"voice file size {size}", flush=True)
        
        if self.file_size_alert <= size < 1048620:
            print(f"***** ALERT FILE SIZE IS {size}")
        elif size == 1048620:
            raise Exception("too big file! you have to store records more frequently")
            
        # empty wav file
        if size != 44:
            shutil.copy(self.voice_file, os.path.join(self.csgo_path, self.output_folder, f'voice_{self.counter}.wav'))
            self.counter += 1
        else:
            print("** empty voice file, skip")
            
        os.remove(self.voice_file)
        self.last_file_create_time = time.time()

        
    def file_is_small_yet(self):
        if not pathlib.Path(self.voice_file).is_file():
            return True
        
        size = os.path.getsize(self.voice_file)
        
        if self.prev_file_size != size:
            print(f"current file size {size}", flush=True)
            self.prev_file_size = size
            
        return size <= self.file_size_alert
    
    
    def record_one_more_file(self):
        
        # before start
        # open csgo, load demo, put to pause (F5)
        
        print(f'=== generating voice file {self.counter}', flush=True)
        print('recording...', flush=True)
        self.fast_forward()
        self.start_record()
        self.resume()
        
#         while(self.file_is_small_yet()):
#             continue

        time.sleep(self.parsing_time)
            
        self.normal_speed()
#         time.sleep(0.02)
        self.pause()
        self.stop_record()
        
    
    def run(self):
        while True:
            self.record_one_more_file()
            if (time.time() - self.last_file_create_time) >= 20:
                break
        
        
    def monitor(self):
        while(self.file_is_small_yet()):
            continue

In [174]:
parser = Parser(csgo_path, parsing_time=0.3)

In [176]:
parser.empty_output_folder()
parser.run()